In [ ]:
## imports

import requests
import pandas as pd
import matplotlib.pyplot as plt
import copy

import pypsa


### Define OPSD Link for data

In [ ]:
# You can find the latest time series data available in the following link: https://data.open-power-system-data.org/time_series/latest/
opsd_url = "https://data.open-power-system-data.org/time_series/2020-10-06/time_series_15min_singleindex.csv"


### Download data from OPSD 

In [1]:
# let's use requests to download the link

try:
    response = requests.get(opsd_url)
    response.raise_for_status()  # Check if the request was successful
    # Save the content to a local CSV file
    with open("opsd_time_series.csv", "wb") as file:
        file.write(response.content)
    print("Download successful! Data saved as 'opsd_time_series.csv'.")
except Exception as e:
    print("An error occurred while downloading the data:")
    print(e)


An error occurred while downloading the data:
name 'requests' is not defined


### Create DF from the loaded data

In [ ]:
try:
    df = pd.read_csv("opsd_time_series.csv", parse_dates=["utc_timestamp"], index_col="utc_timestamp")
    print("Data loaded! Here are the first few rows:")
    print(df.head())
except Exception as e:
    print("An error occurred while loading the CSV file:")
    print(e)

### Check the available countries and load a country data


In [ ]:
print(df.columns.tolist())

load_column = "NL_load_actual_entsoe_transparency"


### Loading column from DF

In [ ]:
if load_column in df.columns:
    # Extract the load data and convert it to an hourly resolution (averaging 15-min data if needed)
    load_data = df[load_column]
    # Resample to hourly mean because many energy system models, including PyPSA, operate on an hourly time resolution. 
    load_data_hourly = load_data.resample("H").mean()  
    print("Sample of hourly load data:")
    print(load_data_hourly.head())
else:
    print(f"Column '{load_column}' not found. Please check the available columns.")

### Define & Create PyPSA network components

In [ ]:
# Create a new PyPSA network
network = pypsa.Network()

# 1. Add a bus (the central node of our grid)
network.add("Bus", "electricity_bus")

# 2. Add a load using the processed hourly data
network.add("Load", "demand", bus="electricity_bus", p_set=load_data_hourly)

# 3. Add a grid connection: an expandable generator representing the external grid
network.add("Generator", "grid", bus="electricity_bus", p_nom_extendable=True, marginal_cost=50)

# 4. Add a renewable generator with dummy parameters
network.add("Generator", "renewable", bus="electricity_bus", p_nom=100, p_max_pu=0.5, marginal_cost=0)

# 5. Add a battery storage unit with example parameters
network.add("StorageUnit",
            "battery",
            bus="electricity_bus",
            p_nom=50,             # Battery power capacity (MW)
            max_hours=4,          # Storage duration (hours)
            efficiency_store=0.95,
            efficiency_dispatch=0.95,
            capital_cost=200)     # Example capital cost (€ per MW)

print("Network components added:")
print("Buses:", network.buses.index.tolist())
print("Loads:", network.loads.index.tolist())
print("Generators:", network.generators.index.tolist())
print("Storage Units:", network.storage_units.index.tolist())

### Define and Run the PyPSA optimization

In [ ]:
# Assuming 'load_data_hourly' is your hourly load series from Step 2
# and you have already created your PyPSA network with the necessary components

# Set the snapshots of the network to match your time series index
network.set_snapshots(load_data_hourly.index)

# Run the linear optimal power flow (LOPF) optimization
# This minimizes the total system cost while meeting demand and respecting all constraints.
network.lopf(network.snapshots)

# Print a summary of the optimization result
print("Objective value (total cost):", network.objective)

# Display the dispatch (power output/input) for the grid connection
print("\nGrid Dispatch (first 5 hours):")
print(network.generators_t.loc[:, "grid"].head())

# Display the battery storage dispatch (positive for discharging, negative for charging)
print("\nBattery Dispatch (first 5 hours):")
print(network.storage_units_t.dispatch.loc[:, "battery"].head())

### Plotting the network & batteries

In [ ]:
# 1. Plot Grid Dispatch
fig, ax = plt.subplots(figsize=(10, 6))
network.generators_t["grid"].plot(ax=ax, label="Grid Dispatch", color="blue")
ax.set_title("Grid Dispatch Over Time")
ax.set_xlabel("Time")
ax.set_ylabel("Power (MW)")
ax.legend()
plt.show()

# 2. Plot Battery Dispatch
fig, ax = plt.subplots(figsize=(10, 6))
# Battery dispatch: positive values indicate discharging; negative, charging.
network.storage_units_t.dispatch["battery"].plot(ax=ax, label="Battery Dispatch", color="orange")
ax.set_title("Battery Dispatch Over Time")
ax.set_xlabel("Time")
ax.set_ylabel("Power (MW)")
ax.legend()
plt.show()

# 3. Plot Load Profile (and Renewable Generation if available)
fig, ax = plt.subplots(figsize=(10, 6))
# Plot load data (demand)
network.loads_t["demand"].plot(ax=ax, label="Load (Demand)", color="red")
# Optionally, if you have renewable generation added:
if "renewable" in network.generators.index:
    network.generators_t["renewable"].plot(ax=ax, label="Renewable Generation", color="green")
ax.set_title("Load and Renewable Generation Over Time")
ax.set_xlabel("Time")
ax.set_ylabel("Power (MW)")
ax.legend()
plt.show()

# 4. Print Objective Value and a Summary of Dispatch
print("Objective Value (Total System Cost):", network.objective)
print("\nGrid Dispatch (first 5 snapshots):")
print(network.generators_t.loc[:, "grid"].head())
print("\nBattery Dispatch (first 5 snapshots):")
print(network.storage_units_t.dispatch.loc[:, "battery"].head())

### Sensitivity Analysis and Model Refinement

In [ ]:

# Define a range of battery capacities (in MW) to test
battery_capacities = [10, 20, 50, 100]  # example capacities
objectives = []

# Save the original network to reset between scenarios
original_network = copy.deepcopy(network)

for capacity in battery_capacities:
    # Create a copy of the original network for each scenario
    test_network = copy.deepcopy(original_network)
    
    # Update the battery capacity for the test scenario
    test_network.storage_units.loc["battery", "p_nom"] = capacity
    
    # Run the optimization (LOPF) over the same snapshots
    test_network.lopf(test_network.snapshots)
    
    # Append the objective value (total system cost)
    objectives.append(test_network.objective)
    print(f"Battery Capacity: {capacity} MW, Objective Value: {test_network.objective:.2f}")

# Plot the sensitivity analysis results
plt.figure(figsize=(8, 5))
plt.plot(battery_capacities, objectives, marker="o", linestyle="-")
plt.xlabel("Battery Capacity (MW)")
plt.ylabel("Objective Value (Total System Cost)")
plt.title("Sensitivity Analysis: Impact of Battery Capacity")
plt.grid(True)
plt.show()